In [71]:
import os
import regex
import glob
import multiprocessing as mp
import importlib
import pprint
import json

from typing import Tuple, Callable, Any, NoReturn, List, Dict, Optional, Union

from functools import partial, reduce

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

from pdf2image import convert_from_path, convert_from_bytes
import matplotlib.pyplot as plt
import cv2 as cv
import pytesseract

In [72]:
import timing
importlib.reload(timing)
import timing

In [73]:
# Instantiate a multiprocess pool.
pool = mp.Pool()

In [112]:
@timing.time_log()
def images_to_strings(x) -> Dict[str, List[str]]:
    """ Envoltura para generar un diccionario del tipo :
            {
                "nombre del archivo PDF": [Lista de strings, uno por página]
            }
        
        A partir de un diccionario del tipo:
            { 
                "nombre del achivo PDF": [Lista contendiendo una imagen por página]
            }
            
        
    """
    
    return {
        nombre: pool.map(pytesseract.image_to_string, archivo) for nombre, archivo in x.items()
    }
##

@timing.time_log()
def tab_by_regex(x: str) -> str:
    """
    """
    
    newline = lambda x: f"{x}\n"
    ntab    = lambda n, txt: n*"\t" + txt
    
    _x_lines: List[str] = x.split('\n')
    _my_str:  str       = ''
        
    for line in _x_lines:
        # Match lines starting with Caps, separated by spaces or points : 
        # r"^([A-Z\s|[A-Z]\.?)+?(?=(\..+))" # found on a separate cell down
        # r"^([A-Z\s]|[A-Z]\.?)+?(?=(\..+))" # the legacy regex for this func
        for i in regex.finditer(r"^([A-Z\s|[A-Z]\.?)+?(?=(\..+))", line): 
            _my_str += newline(i.string)
            _my_str += newline(ntab(1, i.group()))
            # Match numbers, which could be either integers or floats : 
            for j in regex.finditer(r"(\d+\.\d+|\d+)", i.string): # find groups of numbers
                _my_str += newline(ntab(2, j.group()))
            # Match strings found between numbers :
            for k in regex.finditer(r"(?<=(\d+\.\d+|\d+))\D[^\d]+?(?=\s)", i.string):
                _my_str += newline(ntab(3, k.group()))
    
    return _my_str
##

@timing.time_log()
def dict_from_regex(x: str) -> str:
    """
        Genera un directorio con los valores deseado
    """
    
    _x_lines: List[str] = x.split('\n')
    
    _dict2 = {}
    for line in _x_lines:
        for i in regex.finditer(r"^([A-Z\s|[A-Z]\.?)+?(?=(\..+))", line): 
            i.group()
            _valores = []
            _unidades = []
            # Match numbers, which could be either integers or floats : 
            for j in regex.finditer(r"(\d+\.\d+|\d+)", i.string):
                _valores.append(float(j.group()))
            # Match strings found between numbers :
            for k in regex.finditer(r"(?<=(\d+\.\d+|\d+))\D[^\d]+?(?=\s)", i.string):
                _unidades.append(k.group())
            if len(_valores) < 5 and len(_unidades) < 5:
                _dict2.update({
                    i.group(): {
                        "values": _valores,
                        "units": _unidades
                    }
                })
    
    return _dict2
##

@timing.time_log()
def save_results(x: Dict[str, List[str]]) -> bool:
    """
        Guarda los strings a los cuales ya se les han aplicado los filtros.
            
        El modo de escritura es 'w', o sea 'write'.
        Cada vez que se llame esta función los resultados anteriormente guardados 
        serán sobreescritos por los nuevos generados.
        
        La función crea un directorio llamado 'segmented' si es que 
        éste no existe.
        
        Después guarda todos los archivos en dicho directorio.
        
        Regresa:
            'True',  si se logran generar los archivos.
            
            'False', si ocurre algún error.
    """
    
    try:
        if 'segmented' not in os.listdir('.'):
            os.mkdir('segmented')
        
        _camino_resultados = os.path.abspath('segmented')
        _archivos =  list(x.keys())
    
        for i, archivo in enumerate(_archivos):
            for j, page in enumerate(x[archivo]):
                with open(os.path.join(_camino_resultados, f"{_archivos[i].replace('.pdf', '')}.{j+1}.txt"), 'w') as f:
                    f.write(tab_by_regex(page))
    
        return True
    except:
        return False
##                                    

@timing.time_log()
def extract_dates(
    x: Dict[str, List[str]], 
    exclude_date: Optional[str] = None
) -> Dict[str, List[str]]:
    """
        Busca las fechas en un diccionario del siguiente formato :
            { str: Dict[str, List[str]] }
            
        Genera un diccionario con las mismas llaves, pero con una lista 
        de las fechas que encontró en las hojas (cada uno de los strings 
        dentro de la lista).

    """
    
    dates = {}
    for nombre, lista in x.items():
        _tmp_list = []
        for string in lista:
            _tmp_list += list(set(
                regex.findall(r"(\d{2}\D[A-Z]{3}\D\d{4}|\d{2}\D\d{2}\D\d{4})", string)
            ))
        dates.update({
            nombre: list(set(_tmp_list))
        })

    if exclude_date is not None:
        for nombre, lista in dates.items():
            dates[nombre] = list(
                filter(lambda x: x if x != exclude_date else False, lista)
            )
    
    return dates
##
                                       
@timing.time_log()
def save_to_jsonl(x: Dict[str, List[str]], filename: Optional[str] = None) -> bool:
    """
        Guarda los resultados en un archivo, cuyo nombre puede 
        ser opcionalmente especificado a través del parámetro 'filename'.
        
        El NOMBRE por defecto es :
            'resultados.jl'
            
        El nombre del archivo 'filename' puede ser un camino absoluto, si
        se desea guardar la información en algún otro lugar.
        
        POR DEFECTO se trata del mismo directorio :
            En la línea de comando :
                `pwd`
                
            Desde Python :
                os.path.abspath('.')
        
        El formato de registro es JSON-lines :
            http://jsonlines.org/
            
        El modo de edición del archivo es 'append', 
        es decir los nuevos registros se añaden, sin borrar 
        las líneas anteriores.
        
        Prámetros :
                   x: Diccionario { str: List[str] }, la información por guardar.
            filename: Opcional, str, nombre del archivo de registro.
    """
    
    try:
        _archivos = x.keys()
        if filename is None:
            filename = 'resultados.jl'
                                       
        with open(filename, 'a') as f:
            for archivo in _archivos:
                for hoja in x[archivo]:
                    f.write(f"{json.dumps(dict_from_regex(hoja))}\n")
        return True
    except:
        return False
##

@timing.time_log()                                     
def build_string_from_dict(x: Union[Dict[str, List[str]], List[Dict[str, Any]]]) -> str:
    ''' 
        Construye una cadena de caracteres a partir de una de
        dos posibles estructuras anidadas :
        
        1. Diccionario :  {   str: List[str] }
        2. Lista       :  [ { str: Any }     ]
        
        Esta cadena de caracteres es óptima para la visualización.
    '''
    newline = lambda x: f"{x}\n"
    ntab = lambda n, txt: n*"\t" + txt
                                       
    my_string = ''
    
    if type(x) is list:
        for entry in x:
            for key in entry.keys():
                my_string += f'{key}: {entry[key]}\n'
            my_string += '\n\n'
    elif type(x) is dict:
        for key1, value in x.items():
            my_string += f"{key1}:\n"
            for key2 in value.keys():
                my_string += f'\t{key2}: {value[key2]}\n'
            my_string += '\n\n'
                                       
    
    return my_string
##

In [113]:
type({}) is dict

True

In [114]:
newline = lambda x: f"{x}\n"
ntab = lambda n, txt: n*"\t" + txt

In [115]:
path = os.path.abspath('analisis_clinicos/')
path

'/Users/gml/Documents/IX/imagenes/FinalDeImagenes/analisis_clinicos'

Aquí se encuentran todos los pdf con análisis clínicos que tenemos.

In [116]:
path_textos = os.path.abspath('textos')
path_textos

'/Users/gml/Documents/IX/imagenes/FinalDeImagenes/textos'

En esta dirección guardaremos todos los textos reconocidos por **pytesseract**.

In [117]:
caminos_textos = glob.glob(f"{path_textos}/*.txt")
caminos_textos.sort()
#caminos_textos

En esta dirección se almacenaron los textos que se extrajeron de los documentos a partir de **pytesseract**. Nótese que al procesar imagen por imagen, y al crear nosotros una imagen por cada página del pdf, los nombres que ahí se encuentran son los originales, sin la extensión pdf, con un punto indicando el nombre de página y con terminación txt.

```mi_archivo.pdf```  => ```mi_archivo.i.txt``` Donde **i** indica el número de página.

In [118]:
caminos = glob.glob(f"{path}/*.pdf")
#caminos

Esta lista contiene el camino hacia cada uno de los PDFs de los cuales se desean extraer los datos.

In [119]:
archivos = [ os.path.split(camino)[1] for camino in caminos]
archivos

['gustavo_maganna_2018-05-13.pdf',
 'gustavo_maganna_2018-05-06.pdf',
 'gustavo_maganna_2019-11-13.pdf',
 'Wed Nov 13 18:05:58 CST 2019.pdf',
 'InformeResultados1-110600.pdf',
 'gustavo_maganna_2018-01-19.pdf']

Nombre de los archivos PDF, sin el camino absoluto dentro del *filesystem*.

In [120]:
nombres = [ archivo.replace('.pdf', '') for archivo in archivos ]
nombres

['gustavo_maganna_2018-05-13',
 'gustavo_maganna_2018-05-06',
 'gustavo_maganna_2019-11-13',
 'Wed Nov 13 18:05:58 CST 2019',
 'InformeResultados1-110600',
 'gustavo_maganna_2018-01-19']

Nombre de los archivos, sin la extensión ```.pdf```.

In [121]:
ahora_si = { 
    archivo: glob.glob(os.path.join(path_textos ,f"{nombre}.?.txt")) 
    for archivo, nombre in zip(archivos, nombres)
}

In [122]:
[ ahora_si[key].sort() for key in ahora_si.keys() ]
#ahora_si

[None, None, None, None, None, None]

In [123]:
parse_from_txt = True

if parse_from_txt:
    # Buscamos todos los archivos de texto :
    #   los nombres de archivo especificados por 'nombres'
    #   en el directorio especificado por 'path_textos'
    archivos_de_texto = { 
        archivo: glob.glob(os.path.join(path_textos ,f"{nombre}.?.txt")) 
        for archivo, nombre in zip(archivos, nombres)
    }
    # Los ordenamos, para que el orden de las páginas sea el mismo 
    # que en los PDFs :
    [ archivos_de_texto[key].sort() for key in archivos_de_texto.keys() ]
    
    # Creamos un diccionario vacío el cual contendrá todas las cadenas 
    # de acaracteres asociadas a cada una de las hojas de cada uno de los reportes
    # de análisis clínicos.
    strings = {}
    
    # Iteramos sobre cada uno de los nombres de archivo pdf
    for key in archivos_de_texto.keys():
        # Generamos una lista vacía para contener las 'n'
        # cadenas de caracteres, correspondientes a las 'n'
        # páginas de cada pdf.
        _strings = []
        for _file in archivos_de_texto[key]:
            # Iteramos sobre cada 'hoja' perteneciente al pdf especificado.
            with open(_file, 'r') as f:
              _strings.append(f.read())
        strings.update({
            # Añadimos el archivo como llave y 
            # la lista de strings como valor al diccionario.
            key: _strings
        })

In [124]:
# Obtain text from the PDFs, directly (this takes about a minute) : 
from_pdfs = False

if from_pdfs:
    # Para acelerar el proceso, generamos imágenes de forma paralela
    # gracias a pool.map :
    imagenes = pool.map(convert_from_path, caminos)
    # Construimos un diccionario con los nombres de archivos y las
    # imágenes obtenidas : 
    archivos_en_imagenes = {
        archivo: imagen for archivo, imagen in zip(archivos, imagenes)
    }
    # Generamos un nuevo diccionario, ahora con cadenas de caracteres
    # en lugar de imágenes, estas primeras obtenidas a través de 
    # pytesseract.
    # la función images_to_strings() utiliza a su vez pool.map, para
    # acelerar el proceso de extracción de caracteres.
    strings = images_to_strings(archivos_en_imagenes)

In [125]:
# Guardamos cada uno de los strings generados, 
# para poder posteriormente acceder a ellos sin necesidad de 
# repetir el procesamiento de imágenes, lo que conforme 
# crezca la base de datos del PDF, se hará más lento.
save = True

if save:
    for nombre, hojas in strings.items():
        for i, hoja in enumerate(hojas):
            _file_name = os.path.join(path_textos, f"{nombre.replace('.pdf', '')}.{i+1}.txt")
            with open(_file_name, "w") as f:
                f.write(hoja)

In [126]:
extract_dates(strings, exclude_date='06/08/1996')

{'gustavo_maganna_2018-05-13.pdf': ['13/05/2018'],
 'gustavo_maganna_2018-05-06.pdf': [],
 'gustavo_maganna_2019-11-13.pdf': ['13-NOV-2019'],
 'Wed Nov 13 18:05:58 CST 2019.pdf': ['13-NOV-2019'],
 'InformeResultados1-110600.pdf': ['15-NOV-2019'],
 'gustavo_maganna_2018-01-19.pdf': ['19-ENE-2018']}

Esta función que creamos ```extract_date()``` permite obtener fechas encontradas en un PDF, con flexibilidad en cuanto a formatos y la posibilidad de excluir una fecha especificada, que bien podría ser la fecha de cumpleaños del paciente.

In [127]:
print(tab_by_regex(strings[archivos[1]][0]))

ARANDA DE LA PARRA DIR. (477) 719-71-02 CONM. 719-71-00 EXT. 143 Y 145 TEL/FAX: 713-33-03
	ARANDA DE LA PARRA DIR
		477
		719
		71
		02
		719
		71
		00
		143
		145
		713
		33
		03
			 CONM.
			 EXT.
			 Y
			 TEL/FAX:
HEMOGLOBINA........0ccccceeeeeeeeeeeee 16.9 g/dL t 14.10 - 16.30
	HEMOGLOBINA.
		0
		16.9
		14.10
		16.30
			ccccceeeeeeeeeeeee
			 g/dL
			 -
HEMATOCRITO. 48.0 % tT 41-47
	HEMATOCRITO
		48.0
		41
		47
			 %
V.C.M.... 86.2 fL 84 - 94
	V.C.M.
		86.2
		84
		94
			 fL
			 -
LEUCOCITOS.......cccccecceecneeeeeeeeee 5.9 Mil/microL 5.30 - 9.50
	LEUCOCITOS.
		5.9
		5.30
		9.50
			 Mil/microL
			 -
PLAQUETAS.......0ccccccceeee tsetse 163 Mil/microL 150 - 450
	PLAQUETAS.
		0
		163
		150
		450
			ccccccceeee
			 Mil/microL
			 -



Aquí observamos cómo la función que definimos ```tab_by_regex()``` puede identficar correctamente los parámetros de interés, aunque incluye datos que pueden no ser los deseados ya que el reconocimiento de caracteres de **pytesseract** no es perfecto.

Esto se deberá tomar en cuenta el momento de generar los registros.

In [128]:
ejemplo = dict_from_regex(strings['gustavo_maganna_2018-01-19.pdf'][0])
print(build_string(ejemplo))

DEPURACION DE CREATININA.:
	values: [133.6, 75.0, 115.0]
	units: [' mL/min']


CREATININA.:
	values: [0.92, 0.61, 1.24]
	units: [' mg/dL', ' -']


CREATININA URINARIA:
	values: [82.6, 39.0, 259.0]
	units: [' mg/dL', ' -']


SUP:
	values: [1.86, 2.0]
	units: []


MICROALBUMINURIA.:
	values: [10.0, 0.0, 100.0]
	units: [' mg/L']





In [129]:
save_to_jsonl(strings)

True

In [130]:
guarda(strings)

True

In [131]:
nombres

['gustavo_maganna_2018-05-13',
 'gustavo_maganna_2018-05-06',
 'gustavo_maganna_2019-11-13',
 'Wed Nov 13 18:05:58 CST 2019',
 'InformeResultados1-110600',
 'gustavo_maganna_2018-01-19']

In [95]:
_file = archivos[1]
print(_file)
print(len(strings[_file]))
print(strings[_file][2])

gustavo_maganna_2018-05-06.pdf
5
LABORATORIO DE PATOLOGIA CLINICA

HOSPITAL AVENIDA HIDALGO No 329 C.P. : 37000 LEON, GTO.
ARANDA DE LA PARRA DIR. (477) 719-71-02 CONM. 719-71-00 EXT. 143 Y 145 TEL/FAX: 713-33-03
TODA LA VIDA CONTIGO
Paciente: GUSTAVO MAGANA LOPEZ Edad: 21 ANOS Sexo: MASCULINO — Cuarto — Cama:
Expediente: Folio: 028028 Seccion: EXTERNO
Fecha Nac.:06/08/1996 Fecha ingreso: 6-MAY-2018 09:20 AM_ Fecha—Hora ler Impresion:6-MAY-2018 01:34 PM

Médico:JUAN CARLOS FERRER SERRANO Procedencia: H. ARANDA DE LA

Ultima Impresion: 6-MAY-2018 07:58 PM

 

 

PARRA
. U. de :
Estudio Resultado Medida Intervalo de Referencia
PERFIL DE LIPIDOS
COLESTEROL....sssssssssssssssesssssssseeeeees 118 mg/dL | 140 - 200 v
< 150 Normal
TRIGLICERIDOS ....csscsscssessenseesee 50 mg/dL 300-500 Noe v
>500 Muy Alto
VLDL COLESTEROL.....s-:ssccssssssseeeeeee 10 mg/dL 5-35
< 100 Optimo |
LDL COLESTEROL...sscssssssssessssesenee 72 mg/dL 130. 169 Lenwnte ato v
160 - 189 Alto
>= 190 Muy Alto
INDICE DE RIESGO

In [97]:
foo = strings[_file][2]

# Split by line breaks :
foo_lines = foo.split("\n")

# Select valid lines (i.e. longer than 5 characters) :
foo_lines = [ line for line in foo_lines ] # if len(line) > 1 ]

#regex.compile(r'[A-Z]')
print(foo_lines[10])
print(foo_lines[14])
"""
for line in foo_lines:
    for i in regex.finditer(r"[0a-z]{2,}", line):
        print(foo_lines[14])
        #print(i.string)
        #print('\t', i.group())
"""

'\nfor line in foo_lines:\n    for i in regex.finditer(r"[0a-z]{2,}", line):\n        print(foo_lines[14])\n        #print(i.string)\n        #print(\'\t\', i.group())\n'

In [98]:
#print(foo)

In [99]:
#print(tab_by_regex(strings[archivos[2]][0]))

In [100]:
nombres

['gustavo_maganna_2018-05-13',
 'gustavo_maganna_2018-05-06',
 'gustavo_maganna_2019-11-13',
 'Wed Nov 13 18:05:58 CST 2019',
 'InformeResultados1-110600',
 'gustavo_maganna_2018-01-19']

In [101]:
archivos

['gustavo_maganna_2018-05-13.pdf',
 'gustavo_maganna_2018-05-06.pdf',
 'gustavo_maganna_2019-11-13.pdf',
 'Wed Nov 13 18:05:58 CST 2019.pdf',
 'InformeResultados1-110600.pdf',
 'gustavo_maganna_2018-01-19.pdf']

In [102]:
camino_resultados = os.path.abspath('segmented')
camino_resultados

'/Users/gml/Documents/IX/imagenes/FinalDeImagenes/segmented'

In [48]:
save_results(strings)

True

In [103]:
os.mkdir('hue')

FileExistsError: [Errno 17] File exists: 'hue'

In [104]:
for i, archivo in enumerate(archivos):
    for j, page in enumerate(strings[archivo]):
        with open(os.path.join(camino_resultados, f"{nombres[i]}.{j+1}.txt"), 'w') as f:
            f.write(tab_by_regex(page))

In [105]:
newline = lambda x: f"{x}\n"
ntab = lambda n, txt: n*"\t" + txt

In [106]:
for archivo in archivos:
    for hoja in strings[archivo]:
        print(dict_from_regex(hoja))

{'COL': {'values': [], 'units': []}, 'DRA': {'values': [], 'units': []}}
{'AV': {'values': [311.0], 'units': []}, 'COL': {'values': [], 'units': []}, 'DRA': {'values': [], 'units': []}}
{'COL': {'values': [], 'units': []}, 'POBLACION ABIERTA': {'values': [], 'units': []}}
{'HEMATOCRITO': {'values': [48.0, 41.0, 47.0], 'units': [' %']}, 'V': {'values': [86.2, 84.0, 94.0], 'units': [' fL', ' -']}, 'LEUCOCITOS': {'values': [5.9, 5.3, 9.5], 'units': [' Mil/microL', ' -']}}
{'GLUCOSA': {'values': [68.0, 70.0, 100.0], 'units': [' mg/dL', ' -']}, 'CREATININA': {'values': [0.76, 0.61, 1.24], 'units': [' mg/dL', ' -']}, 'ACIDO URICO': {'values': [4.9, 4.8, 8.7], 'units': [' mg/dL', ' -']}, 'HEMOGLOBINA GLICOSILADA': {'values': [6.7, 4.5, 6.3], 'units': [' %', ' -']}}
{'COLESTEROL': {'values': [118.0, 140.0, 200.0], 'units': [' mg/dL', ' -']}, 'TRIGLICERIDOS ': {'values': [50.0, 300.0, 500.0], 'units': [' mg/dL', ' Noe']}, 'VLDL COLESTEROL': {'values': [10.0, 5.0, 35.0], 'units': [' mg/dL']}, 'L

In [27]:
_my_str = ''
for line in foo_lines:
    for i in regex.finditer(r"^([A-Z\s]|[A-Z]\.?)+?(?=(\..+))", line): # FIND lines starting with Caps 
        _my_str += newline(i.string)
        _my_str += newline(ntab(1, i.group()))
        for j in regex.finditer(r"(\d+\.\d+|\d+)", i.string): # find groups of numbers
            _my_str += newline(ntab(2, j.group()))
        for k in regex.finditer(r"(?<=(\d+\.\d+|\d+))\D[^\d]+?(?=\s)", i.string):
            _my_str += newline(ntab(3, k.group()))

In [28]:
print(_my_str)

ARANDA DE LA PARRA DIR. (477) 719-71-02 CONM. 719-71-00 EXT. 143 Y 145 TEL/FAX: 713-33-03
	ARANDA DE LA PARRA DIR
		477
		719
		71
		02
		719
		71
		00
		143
		145
		713
		33
		03
			 CONM.
			 EXT.
			 Y
			 TEL/FAX:
COLESTEROL....sssssssssssssssesssssssseeeeees 118 mg/dL | 140 - 200 v
	COLESTEROL
		118
		140
		200
			 mg/dL
			 -
TRIGLICERIDOS ....csscsscssessenseesee 50 mg/dL 300-500 Noe v
	TRIGLICERIDOS 
		50
		300
		500
			 mg/dL
			 Noe
VLDL COLESTEROL.....s-:ssccssssssseeeeeee 10 mg/dL 5-35
	VLDL COLESTEROL
		10
		5
		35
			 mg/dL
LDL COLESTEROL...sscssssssssessssesenee 72 mg/dL 130. 169 Lenwnte ato v
	LDL COLESTEROL
		72
		130
		169
			 mg/dL
			 Lenwnte
HDL COLESTEROL.......csssccsssseeeesseee 33 mg/dL IEERMEDIO. cases v
	HDL COLESTEROL
		33
			 mg/dL
ALTO... MENOR A 35,
	ALTO
		35
LIPIDOS TOTALES......:ssssssscsseeesssesseees 337 mg/dL | 450 - 1000
	LIPIDOS TOTALES
		337
		450
		1000
			 mg/dL
			 -
INDICE ATEROGENICO... 3.6 RiEScO INTERMEDIO: 4.5 -8
	INDICE ATEROGENICO
		3.6

In [31]:
"""
for line in foo_lines:
    for i in regex.finditer(r"^([A-Z\s]|[A-Z]\.?)+?(?=(\..+))", line): # FIND lines starting with Caps 
        print(i.string)
        print('\t',i.group())
        for j in regex.finditer(r"(\d+\.\d+|\d+)", i.string): # find groups of numbers
            print(2*'\t',j.group())
        for k in regex.finditer(r"(?<=(\d+\.\d+|\d+))\D[^\d]+?(?=\s)", i.string):
            print(3*'\t',k.group())
"""

'\nfor line in foo_lines:\n    for i in regex.finditer(r"^([A-Z\\s]|[A-Z]\\.?)+?(?=(\\..+))", line): # FIND lines starting with Caps \n        print(i.string)\n        print(\'\t\',i.group())\n        for j in regex.finditer(r"(\\d+\\.\\d+|\\d+)", i.string): # find groups of numbers\n            print(2*\'\t\',j.group())\n        for k in regex.finditer(r"(?<=(\\d+\\.\\d+|\\d+))\\D[^\\d]+?(?=\\s)", i.string):\n            print(3*\'\t\',k.group())\n'

In [33]:
# My regexps :

my_may_ge_2 = r"^[A-Z]{2,}"  # Find CAPITAL WORDS longer than 2, at the begining of the line.
my_numbers = r"(\d+\.\d+|\d+)" # Find numbers of any length, with a decimal point or not.
my_from_begining_until_point = r"^.+?(?=(\..+))" # Find any text before the first occurrence of a pint
my_caps_and_whitespace_until_point = r"^[A-Z\s]+?(?=(\..+))"
my_caps_and_whitspace_or_acronym_until_point = r"^([A-Z\s|[A-Z]\.?)+?(?=(\..+))"
my_failed_extract_units = r"(?<=((\d+\.\d+|\d+)+))(.+)"
googled_text_between_brackets = r"(?<=\[).+?(?=\])"
my_extract_units = r"(?<=(\d+\.\d+|\d+))\D+?(?=(\d+\.\d+|\d+))"
my_better_extract_units_between_numbers = r"(?<=(\d+\.\d+|\d+))\D[^\.\d]+?(?=(\d+\.\d+|\d+))"
my_extract_units_between_numbers_and_whitespace = r"(?<=(\d+\.\d+|\d+))\D[^\.\d]+?(?=\s)"

In [92]:
dir(regex)
help(regex.purge)

Help on function purge in module regex.regex:

purge()
    Clear the regular expression cache



In [93]:
for line in foo_lines:
    for i in regex.findall(r"^[A-Z]{2,}", line):
        print(i)

NameError: name 'foo_lines' is not defined

In [101]:
x = "\n hola \n"

In [103]:
r"{}".format(x)

'\n hola \n'

In [99]:
print(r'\n\nhola')

\n\nhola


In [74]:
with open('watever', 'w') as lol:
    lol.write(strings[archivos[2]][0])

In [78]:
# Cuántas líneas tenemos en total :
caracteres_por_linea = pd.core.series.Series(
    [
     len(line) for line in page.split('\n') 
     for pdf in strings 
     for page in pdf
    ],
    name='caracteres'
)

NameError: name 'page' is not defined

In [76]:
sns.distplot(caracteres)

NameError: name 'caracteres' is not defined

In [77]:
pool.close()
pool.terminate()